<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/ai%2Fcount_vector_model_pre_covid_model/AI_pre_covid_prediction_count_vector_ml_sentiment_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("StopWords").getOrCreate()
spark = SparkSession.builder.appName("Fin-sent-count-vector").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/sentiment-train.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment-train.csv"), sep=",", header=None)



# Show DataFrame
df.show(5, truncate=False)


+--------+-----------------------------------------------------------------------------------------------------+
|_c0     |_c1                                                                                                  |
+--------+-----------------------------------------------------------------------------------------------------+
|Positive|Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                              |
|Positive|Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                     |
|Positive|RSX: Russian Equities Are Still Undervalued                                                          |
|Positive|XLB: Hold Your Position                                                                              |
|Positive|Musgrave Minerals broad, near-surface high-grade results open up further Cue gold discovery potential|
+--------+--------------------------------------------------------------------------------------

In [5]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
|Positive|Autodesk Is Down ...|    71|
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
+--------+--------------------+------+
only showing top 20 rows



In [6]:
df2 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df2.printSchema()
df2.show(20, truncate = False)


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|class   |text                                                                                                                                                                         |length|
+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Positive|Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                                                                                                      |71    |
|Positive|Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                        

In [7]:
df2.createOrReplaceTempView('temp')
df3 = spark.sql('select case class when "Positive" then 1.0  when "Negative" then 2.0 else 0 end as label, text from temp')
df3.show(20, truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|text                                                                                                                                                                         |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1.0  |Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                                                                                                      |
|1.0  |Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                                                                                             |
|1.0  |RSX: Russian Equities Are Still Undervalued                                        

In [8]:
from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(df3)
wordsData.show()

+-----+--------------------+--------------------+
|label|                text|               words|
+-----+--------------------+--------------------+
|  1.0|Autodesk Is Down ...|[autodesk, is, do...|
|  1.0|Blue Shield of Ca...|[blue, shield, of...|
|  1.0|RSX: Russian Equi...|[rsx:, russian, e...|
|  1.0|XLB: Hold Your Po...|[xlb:, hold, your...|
|  1.0|Musgrave Minerals...|[musgrave, minera...|
|  1.0|ViacomCBS to rest...|[viacomcbs, to, r...|
|  1.0|Oil climbs to one...|[oil, climbs, to,...|
|  2.0|Australian regula...|[australian, regu...|
|  1.0|The Wall Street J...|[the, wall, stree...|
|  1.0|Ora Banda Mining ...|[ora, banda, mini...|
|  1.0|Google Pay Plans ...|[google, pay, pla...|
|  2.0|Toyota, Honda Ang...|[toyota,, honda, ...|
|  0.0|ZY Investor Alert...|[zy, investor, al...|
|  0.0|Calima Energy spu...|[calima, energy, ...|
|  1.0|These Marijuana S...|[these, marijuana...|
|  1.0|Moderna ETFs to R...|[moderna, etfs, t...|
|  0.0|MacroGenics Annou...|[macrogenics, ann...|


In [9]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
featurizedData.show()

+-----+--------------------+--------------------+--------------------+
|label|                text|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  1.0|Autodesk Is Down ...|[autodesk, is, do...|(11788,[6,9,19,37...|
|  1.0|Blue Shield of Ca...|[blue, shield, of...|(11788,[2,7,177,5...|
|  1.0|RSX: Russian Equi...|[rsx:, russian, e...|(11788,[48,262,63...|
|  1.0|XLB: Hold Your Po...|[xlb:, hold, your...|(11788,[158,335,1...|
|  1.0|Musgrave Minerals...|[musgrave, minera...|(11788,[26,49,89,...|
|  1.0|ViacomCBS to rest...|[viacomcbs, to, r...|(11788,[0,34,1891...|
|  1.0|Oil climbs to one...|[oil, climbs, to,...|(11788,[0,7,100,1...|
|  2.0|Australian regula...|[australian, regu...|(11788,[5,175,179...|
|  1.0|The Wall Street J...|[the, wall, stree...|(11788,[1,7,292,4...|
|  1.0|Ora Banda Mining ...|[ora, banda, mini...|(11788,[0,5,72,32...|
|  1.0|Google Pay Plans ...|[google, pay, pla...|(11788,[75,217,26...|
|  2.0

In [10]:
from pyspark.ml.feature import  IDF
#IDF down-weighs features which appear frequently in a corpus. This generally improves performance when using text as features since most frequent, and hence less important words, get down-weighed.
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show()  
# We want only the label and features columns for our machine learning models

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(11788,[6,9,19,37...|
|  1.0|(11788,[2,7,177,5...|
|  1.0|(11788,[48,262,63...|
|  1.0|(11788,[158,335,1...|
|  1.0|(11788,[26,49,89,...|
|  1.0|(11788,[0,34,1891...|
|  1.0|(11788,[0,7,100,1...|
|  2.0|(11788,[5,175,179...|
|  1.0|(11788,[1,7,292,4...|
|  1.0|(11788,[0,5,72,32...|
|  1.0|(11788,[75,217,26...|
|  2.0|(11788,[2,65,230,...|
|  0.0|(11788,[43,44,63,...|
|  0.0|(11788,[10,83,338...|
|  1.0|(11788,[0,3,13,18...|
|  1.0|(11788,[0,7,172,2...|
|  0.0|(11788,[11,12,13,...|
|  0.0|(11788,[1,3,5,11,...|
|  0.0|(11788,[0,2,3,12,...|
|  0.0|(11788,[0,4,12,13...|
+-----+--------------------+
only showing top 20 rows



In [11]:
seed = 0  # set seed for reproducibility
trainDF, testDF = rescaledData.randomSplit([0.8,0.2],seed)

In [12]:
trainDF.count()

3965

In [17]:
trainDF.show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
testDF.count()

1035

In [15]:
testDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+
|label|                text|               words|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+--------------------+
|  0.0|180 Life Sciences...|[180, life, scien...|(11788,[10,12,197...|(11788,[10,12,197...|
|  0.0|1847 Goedeker Iss...|[1847, goedeker, ...|(11788,[45,87,474...|(11788,[45,87,474...|
|  0.0|3 Reasons to Buy ...|[3, reasons, to, ...|(11788,[0,4,23,54...|(11788,[0,4,23,54...|
|  0.0|3 Things to Watch...|[3, things, to, w...|(11788,[0,1,3,8,3...|(11788,[0,1,3,8,3...|
|  0.0|3 Upcoming Earnin...|[3, upcoming, ear...|(11788,[0,19,54,6...|(11788,[0,19,54,6...|
|  0.0|4 Diversified Che...|[4, diversified, ...|(11788,[0,5,18,17...|(11788,[0,5,18,17...|
|  0.0|4 Outstanding Sto...|[4, outstanding, ...|(11788,[0,18,20,2...|(11788,[0,18,20,2...|
|  0.0|4 Top Stock Trade...|[4, top, stock, t...|(11788,[5,8,61,17...|(11788,[5,

In [18]:
#Logistic Regression Classifier
#Cross validator different classification to  
#Param Grid apply different parameters, testing which param is best
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
lr = LogisticRegression(maxIter = 10)

paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, np.linspace(0.3, 0.01, 10)) \
    .addGrid(lr.elasticNetParam, np.linspace(0.3, 0.8, 6)) \
    .build()
crossval_lr = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid_lr,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds= 5)  
cvModel_lr = crossval_lr.fit(trainDF)
best_model_lr = cvModel_lr.bestModel.summary
best_model_lr.predictions.columns

['label',
 'text',
 'words',
 'rawFeatures',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [19]:
best_model_lr.predictions.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|                text|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0|"ASSA ABLOY to ac...|["assa, abloy, to...|(11788,[0,2,4,313...|(11788,[0,2,4,313...|[1.44743642329661...|[0.75639685633990...|       0.0|
|  0.0|"BofA Unveils ""V...|["bofa, unveils, ...|(11788,[399,7222,...|(11788,[399,7222,...|[1.38335073169863...|[0.68327611691678...|       0.0|
|  0.0|"CP Offer for KCS...|["cp, offer, for,...|(11788,[5,75,344,...|(11788,[5,75,344,...|[1.14291777705582...|[0.69712278726930...|       0.0|
|  0.0|"Cumulus Podcast ...|["cumulus, podcas...|(11788,[0,2,10,13...|(11788,[0,2,10,13...|[2.37344340370829...|[0.69345229289871.

In [153]:
#Area under the curve for the training data
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_lr = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_lr.evaluate(best_model_lr.predictions)

0.9987181653124033

In [154]:
#F1 score
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lr.evaluate(best_model_lr.predictions)

0.9987389561845692

In [155]:
#Accuracy 
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lr.evaluate(best_model_lr.predictions)

0.9987389659520807

In [20]:
#prediction
train_fit_lr = best_model_lr.predictions.select('label','prediction')
train_fit_lr.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0| 1515|
|  0.0|       1.0|    4|
|  1.0|       0.0|   13|
|  2.0|       2.0|  448|
|  2.0|       1.0|    8|
|  0.0|       0.0| 1975|
|  0.0|       2.0|    2|
+-----+----------+-----+



In [21]:
#import pre-covid test data
url1 = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/pre-covid-test-data.csv'
spark.sparkContext.addFile(url1)
pre_covid_df = spark.read.csv(SparkFiles.get("pre-covid-test-data.csv"), sep=",", header=None)

pre_covid_df.show(5, truncate=False)

+--------------------------------------------------------------------------------------+
|_c0                                                                                   |
+--------------------------------------------------------------------------------------+
|Stitch Fix, Inc. (SFIX) CEO Katrina Lake on Q4 2019 Results - Earnings Call Transcript|
|Goldman Sachs reshuffles Asia M&A leadership as John Kim retires - memo               |
|What We Learned From Amazon's Big Hardware Announcements                              |
|Cerner: A Great Stock To Add To Your Portfolio In 2020                                |
|Johnson & Johnson settles Ohio lawsuits to avoid federal trial                        |
+--------------------------------------------------------------------------------------+
only showing top 5 rows



In [22]:
#Rename column
pre_covid_df2 = pre_covid_df.withColumnRenamed("_c0", "text")
pre_covid_df2.printSchema()
pre_covid_df2.show(20, truncate = False)

root
 |-- text: string (nullable = true)

+--------------------------------------------------------------------------------------+
|text                                                                                  |
+--------------------------------------------------------------------------------------+
|Stitch Fix, Inc. (SFIX) CEO Katrina Lake on Q4 2019 Results - Earnings Call Transcript|
|Goldman Sachs reshuffles Asia M&A leadership as John Kim retires - memo               |
|What We Learned From Amazon's Big Hardware Announcements                              |
|Cerner: A Great Stock To Add To Your Portfolio In 2020                                |
|Johnson & Johnson settles Ohio lawsuits to avoid federal trial                        |
|5 Key Conclusions From United Technologies' Latest Presentation                       |
|Foot Locker: Worth The Gamble                                                         |
|The Stock Market Dived Today, but These 2 Stocks Blasted Higher    

In [23]:
#tokenize text
# from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(pre_covid_df2)
wordsData.show()

+--------------------+--------------------+
|                text|               words|
+--------------------+--------------------+
|Stitch Fix, Inc. ...|[stitch, fix,, in...|
|Goldman Sachs res...|[goldman, sachs, ...|
|What We Learned F...|[what, we, learne...|
|Cerner: A Great S...|[cerner:, a, grea...|
|Johnson & Johnson...|[johnson, &, john...|
|5 Key Conclusions...|[5, key, conclusi...|
|Foot Locker: Wort...|[foot, locker:, w...|
|The Stock Market ...|[the, stock, mark...|
|These High-Yield ...|[these, high-yiel...|
|This High-Yield S...|[this, high-yield...|
|New Oriental Educ...|[new, oriental, e...|
|J&J to pay $20.4 ...|[j&j, to, pay, $2...|
|Pharma Giant J&J ...|[pharma, giant, j...|
|Brokerage Stocks ...|[brokerage, stock...|
|Johnson & Johnson...|[johnson, &, john...|
|Visa, Mastercard ...|[visa,, mastercar...|
|Apple Looking For...|[apple, looking, ...|
|Why Cloud Computi...|[why, cloud, comp...|
|Micron: Back To R...|[micron:, back, t...|
|The New Ireland F...|[the, new,

In [24]:
#count vectorize 
# from pyspark.ml.feature import CountVectorizer
# count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
# model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
featurizedData.show()

+--------------------+--------------------+--------------------+
|                text|               words|         rawFeatures|
+--------------------+--------------------+--------------------+
|Stitch Fix, Inc. ...|[stitch, fix,, in...|(11788,[7,14,19,2...|
|Goldman Sachs res...|[goldman, sachs, ...|(11788,[20,34,487...|
|What We Learned F...|[what, we, learne...|(11788,[29,45,187...|
|Cerner: A Great S...|[cerner:, a, grea...|(11788,[0,3,6,8,1...|
|Johnson & Johnson...|[johnson, &, john...|(11788,[0,28,269,...|
|5 Key Conclusions...|[5, key, conclusi...|(11788,[45,155,25...|
|Foot Locker: Wort...|[foot, locker:, w...|(11788,[1,320,177...|
|The Stock Market ...|[the, stock, mark...|(11788,[1,8,18,55...|
|These High-Yield ...|[these, high-yiel...|(11788,[0,18,48,2...|
|This High-Yield S...|[this, high-yield...|(11788,[1,8,37,84...|
|New Oriental Educ...|[new, oriental, e...|(11788,[4,22,276,...|
|J&J to pay $20.4 ...|[j&j, to, pay, $2...|(11788,[0,10,72,2...|
|Pharma Giant J&J ...|[ph

In [25]:
from pyspark.ml.feature import  IDF
#IDF down-weighs features which appear frequently in a corpus. This generally improves performance when using text as features since most frequent, and hence less important words, get down-weighed.
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("features").show()  
# We want only the label and features columns for our machine learning models

+--------------------+
|            features|
+--------------------+
|(11788,[7,14,19,2...|
|(11788,[20,34,487...|
|(11788,[29,45,187...|
|(11788,[0,3,6,8,1...|
|(11788,[0,28,269,...|
|(11788,[45,155,25...|
|(11788,[1,320,177...|
|(11788,[1,8,18,55...|
|(11788,[0,18,48,2...|
|(11788,[1,8,37,84...|
|(11788,[4,22,276,...|
|(11788,[0,10,72,2...|
|(11788,[3,10,72,2...|
|(11788,[18,20,421...|
|(11788,[2,10,28,2...|
|(11788,[102,192,8...|
|(11788,[5,22,86,1...|
|(11788,[5,6,15,67...|
|(11788,[0,408,261...|
|(11788,[1,14,22,2...|
+--------------------+
only showing top 20 rows



In [147]:
rescaledData1 = rescaledData.drop('features')
rescaledData1.show()

+--------------------+--------------------+--------------------+
|                text|               words|         rawFeatures|
+--------------------+--------------------+--------------------+
|Stitch Fix, Inc. ...|[stitch, fix,, in...|(17680,[0,1,2,3,4...|
|Goldman Sachs res...|[goldman, sachs, ...|(17680,[3,25,167,...|
|What We Learned F...|[what, we, learne...|(17680,[30,36,98,...|
|Cerner: A Great S...|[cerner:, a, grea...|(17680,[7,10,15,1...|
|Johnson & Johnson...|[johnson, &, john...|(17680,[7,55,204,...|
|5 Key Conclusions...|[5, key, conclusi...|(17680,[36,93,165...|
|Foot Locker: Wort...|[foot, locker:, w...|(17680,[9,348,104...|
|The Stock Market ...|[the, stock, mark...|(17680,[9,18,22,3...|
|These High-Yield ...|[these, high-yiel...|(17680,[7,22,51,1...|
|This High-Yield S...|[this, high-yield...|(17680,[9,18,41,6...|
|New Oriental Educ...|[new, oriental, e...|(17680,[17,24,294...|
|J&J to pay $20.4 ...|[j&j, to, pay, $2...|(17680,[7,23,71,1...|
|Pharma Giant J&J ...|[ph

In [148]:
# df_with_x4 = df.withColumn("x4", lit(0))
from pyspark.sql.functions import lit
rescaledData2 = rescaledData.withColumn("label", lit(0))
rescaledData2.show()

+--------------------+--------------------+--------------------+--------------------+-----+
|                text|               words|         rawFeatures|            features|label|
+--------------------+--------------------+--------------------+--------------------+-----+
|Stitch Fix, Inc. ...|[stitch, fix,, in...|(17680,[0,1,2,3,4...|(17680,[0,1,2,3,4...|    0|
|Goldman Sachs res...|[goldman, sachs, ...|(17680,[3,25,167,...|(17680,[3,25,167,...|    0|
|What We Learned F...|[what, we, learne...|(17680,[30,36,98,...|(17680,[30,36,98,...|    0|
|Cerner: A Great S...|[cerner:, a, grea...|(17680,[7,10,15,1...|(17680,[7,10,15,1...|    0|
|Johnson & Johnson...|[johnson, &, john...|(17680,[7,55,204,...|(17680,[7,55,204,...|    0|
|5 Key Conclusions...|[5, key, conclusi...|(17680,[36,93,165...|(17680,[36,93,165...|    0|
|Foot Locker: Wort...|[foot, locker:, w...|(17680,[9,348,104...|(17680,[9,348,104...|    0|
|The Stock Market ...|[the, stock, mark...|(17680,[9,18,22,3...|(17680,[9,18,22,

In [26]:
#Predict using the test data and evaluate the predictions
predictions_lr = cvModel_lr.transform(rescaledData)
predictions_lr.columns

['text',
 'words',
 'rawFeatures',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [27]:
predictions_lr.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Stitch Fix, Inc. ...|[stitch, fix,, in...|(11788,[7,14,19,2...|(11788,[7,14,19,2...|[1.94331532220843...|[0.82131743103819...|       0.0|
|Goldman Sachs res...|[goldman, sachs, ...|(11788,[20,34,487...|(11788,[20,34,487...|[0.73309243034845...|[0.52961788112530...|       0.0|
|What We Learned F...|[what, we, learne...|(11788,[29,45,187...|(11788,[29,45,187...|[0.29297683823490...|[0.35642853939398...|       1.0|
|Cerner: A Great S...|[cerner:, a, grea...|(11788,[0,3,6,8,1...|(11788,[0,3,6,8,1...|[0.81621249887911...|[0.15710229008479...|       1.0|
|Johnson & Johnson...|[john

In [ ]:
rescaledData.show()

In [ ]:
predictions_lr.select('prediction').show()

In [ ]:
predictions_lr.select('features').show()

In [ ]:
predictions_lr.show(5)

In [ ]:
#Show sample predictions:
predictions_lr.select('prediction').show(5)

In [ ]:
#hit vs missed
predictions_lr.groupBy('label','prediction').count().show()

In [ ]:
#Area under the curve with the test data
my_eval_lr.evaluate(predictions_lr)

In [ ]:
#F1-score with the test data
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lr.evaluate(predictions_lr)

In [ ]:
#Acccuracy with the test data
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lr.evaluate(predictions_lr)

In [28]:
#Naive Bayes using MulticlassClassificationEval
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
nb = NaiveBayes()
paramGrid_nb = ParamGridBuilder() \
    .addGrid(nb.smoothing, np.linspace(0.3, 10, 10)) \
    .build()
crossval_nb = CrossValidator(estimator=nb,
                          estimatorParamMaps=paramGrid_nb,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds= 5) 
cvModel_nb = crossval_nb.fit(trainDF)

In [ ]:
#average area under the curve values
cvModel_nb.avgMetrics

In [ ]:
#Make predictions:
predictions_nb = cvModel_nb.transform(testDF)
predictions_nb.select('label', 'prediction').show(30)

In [32]:
#Make predictions:
predictions_nb = cvModel_nb.transform(rescaledData)
predictions_nb.select('text','prediction').show(truncate = False)

+--------------------------------------------------------------------------------------+----------+
|text                                                                                  |prediction|
+--------------------------------------------------------------------------------------+----------+
|Stitch Fix, Inc. (SFIX) CEO Katrina Lake on Q4 2019 Results - Earnings Call Transcript|0.0       |
|Goldman Sachs reshuffles Asia M&A leadership as John Kim retires - memo               |0.0       |
|What We Learned From Amazon's Big Hardware Announcements                              |1.0       |
|Cerner: A Great Stock To Add To Your Portfolio In 2020                                |1.0       |
|Johnson & Johnson settles Ohio lawsuits to avoid federal trial                        |1.0       |
|5 Key Conclusions From United Technologies' Latest Presentation                       |0.0       |
|Foot Locker: Worth The Gamble                                                         |1.0       |


In [36]:
predictions_nb.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Stitch Fix, Inc. ...|[stitch, fix,, in...|(11788,[7,14,19,2...|(11788,[7,14,19,2...|[-166.61065485499...|[0.99999999999998...|       0.0|
|Goldman Sachs res...|[goldman, sachs, ...|(11788,[20,34,487...|(11788,[20,34,487...|[-548.96915241475...|[0.99985353226443...|       0.0|
|What We Learned F...|[what, we, learne...|(11788,[29,45,187...|(11788,[29,45,187...|[-342.46090573197...|[2.21088311028088...|       1.0|
|Cerner: A Great S...|[cerner:, a, grea...|(11788,[0,3,6,8,1...|(11788,[0,3,6,8,1...|[-284.09699146652...|[8.74990887619669...|       1.0|
|Johnson & Johnson...|[john

In [53]:
predictions_nb_df = predictions_nb.select('prediction', 'text')
predictions_nb_df.show(5)

+----------+--------------------+
|prediction|                text|
+----------+--------------------+
|       0.0|Stitch Fix, Inc. ...|
|       0.0|Goldman Sachs res...|
|       1.0|What We Learned F...|
|       1.0|Cerner: A Great S...|
|       1.0|Johnson & Johnson...|
+----------+--------------------+
only showing top 5 rows



In [47]:
type(predictions_nb_df.select('prediction'))

pyspark.sql.dataframe.DataFrame

In [55]:
predictions_nb_df.createOrReplaceTempView('temp')
predictions_nb_df1 = spark.sql('select case prediction when "1.0" then "Positive"  when "2.0" then "Negative" else "Neutral" end as predicted_label, text from temp')
predictions_nb_df1.show(20, truncate = False)

+---------------+--------------------------------------------------------------------------------------+
|predicted_label|text                                                                                  |
+---------------+--------------------------------------------------------------------------------------+
|Neutral        |Stitch Fix, Inc. (SFIX) CEO Katrina Lake on Q4 2019 Results - Earnings Call Transcript|
|Neutral        |Goldman Sachs reshuffles Asia M&A leadership as John Kim retires - memo               |
|Positive       |What We Learned From Amazon's Big Hardware Announcements                              |
|Positive       |Cerner: A Great Stock To Add To Your Portfolio In 2020                                |
|Positive       |Johnson & Johnson settles Ohio lawsuits to avoid federal trial                        |
|Neutral        |5 Key Conclusions From United Technologies' Latest Presentation                       |
|Positive       |Foot Locker: Worth The Gamble         

In [ ]:
#hit vs missed
predictions_nb.groupBy('label','prediction').count().show()

In [ ]:
#Area under ROC
my_eval_nb = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_nb.evaluate(predictions_nb)

In [ ]:
#F1
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_nb.evaluate(predictions_nb)

In [ ]:
#Accuracy
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_nb.evaluate(predictions_nb)